# Load Functions

## 0. imports

In [ ]:
from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.enums import TA_LEFT
import mne

from utils import *
from eeg import compute_eeg_pipeline, test_eeg_pipeline
from ecg_qc import ecg_qc 
from eda_qc import eda_qc
from rsp_qc import *
from mic_qc import *
from lsl_problem import *
from et_qc import *
from webcam_qc import webcam_qc

subject = "P5029423"

xdf_filename = f'/Users/bryan.gonzalez/CUNY_subs/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'
# xdf_filename = f'/Users/camilla.strauss/Desktop/CUNY_Data/Data/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'


In [ ]:
subject = 'P5029423'
xdf_filename = f'/Users/camilla.strauss/Desktop/CUNY_Data/Data/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'

# 1. EEG

In [ ]:
#Compute the eeg vars
eeg_vars = compute_eeg_pipeline(xdf_filename)

In [ ]:
# which version of mne 
import matplotlib
print(matplotlib.__version__)

# 2. ECG

In [ ]:
[ecg_vars, ecg_plt] = ecg_qc(xdf_filename)


# 3. EDA

In [ ]:
[eda_vars, eda_plt1, eda_plt2] = eda_qc(xdf_filename)

# 4. RSP

In [ ]:
rsp_vars = rsp_qc(xdf_filename)

In [ ]:
rsp_vars

# 5. Mic

In [ ]:
mic_vars = mic_qc(xdf_filename)

# 6. Webcam

In [ ]:
webcam_vars = webcam_qc(xdf_filename==xdf_filename,
                video_file=video_filename, task='RestingState')

# 7. ET

In [ ]:
et_vars = et_qc(xdf_filename)

# Stream Durations 

### functions are in utils.py (but should be called in the report)

In [ ]:
get_durations(ExperimentPart='Experiment', xdf_path=xdf_filename)

In [ ]:
get_durations('RestingState', xdf_filename)

In [ ]:
# i wont run these but they are here for reference
# get_durations('CampFriend', xdf_filename)
# get_durations('SocialTask', xdf_filename)

In [ ]:
whole_durations(xdf_filename)

# LSL Problem

In [ ]:
lsl_vars = lsl_problem(xdf_filename)

In [ ]:
pprint(lsl_vars)

# Report

## 1. Set up Document

In [ ]:


# Set up the document
doc = SimpleDocTemplate("example_report.pdf", pagesize=LETTER)
styles = getSampleStyleSheet()
story = []

# Add a title
title = Paragraph(f"Subject Report: {subject}", styles["Title"])
story.append(title)
story.append(Spacer(1, 20))

# Add a paragraph
text = f"""
Collection Date: {get_collection_date(xdf_filename)} 
"""
paragraph = Paragraph(text, styles["BodyText"])
story.append(paragraph)
story.append(Spacer(1, 20))




In [ ]:
# Add a subtitle
subtitle = Paragraph(f"EEG", styles["Heading2"], )
story.append(subtitle)
story.append(Spacer(1, 5))

# Add a paragraph
text = f"""
Data preprocessed by performing <b>line noise removal</b>, <b>robust referencing</b>, and <b>bad channel detection/interpolation</b> using PyPrep pipeline. First, the pipeline applies a notch filter at 60 Hz and its harmonics to remove power line noise. Then, it performs <b>robust average referencing</b>, where it detects bad channels, interpolates them using surrounding signals, and computes a median-based reference across EEG channels. This ensures a stable reference even in the presence of noisy electrodes. The final output is a cleaned EEG dataset with a consistent reference, ready for further analysis.
"""

paragraph = Paragraph(text, styles["BodyText"], )
story.append(paragraph)
story.append(Spacer(1, 20))

In [ ]:
# add an image
from reportlab.lib.units import inch
from reportlab.platypus import Image
image = Image("/Users/bryan.gonzalez/MOBI_QC/src/MOBI_QC/report_images/P5029423_eeg_annotations.png", 7*inch, 3*inch)
image.hAlign = 'CENTER'
story.append(image)

In [ ]:
story.append(Spacer(1, 20))

In [ ]:
# Sample data for the table
data = [
    ["variable", "value"],
    ["bad channels before", str(eeg_vars["bad_channels_before"])],
    ["interpolated channels", str(eeg_vars["interpolated_channels"])],
    ["bad channels after", str(eeg_vars["bad_channels_after"])],
   ["percentage of good data", str(eeg_vars["percent_good"])],
]

In [ ]:



# Define the table style to make all borders white
style = TableStyle([
    ('GRID', (0,0), (-1,-1), 1, colors.white),  # All grid lines white
    ('BOX', (0,0), (-1,-1), 1, colors.white),   # Outer box white
    ('INNERGRID', (0,0), (-1,-1), 1, colors.white)  # Inner grid white
])
# Create the table
table = Table(data, style=style)
# Apply style
#table.setStyle(style)
story.append(table)
#story.append(Spacer(1, 20))

In [ ]:
# Build the PDF
doc.build(story)

In [ ]:
table